In [1]:
#%% 
import numpy as np
import pandas as pd
import pickle as pickle
from common_functions import get_clean_data

In [2]:
def rational_to_double(rat):
    """ Takes a string of typ 4/4 or 4 and returns the corresponding float"""
    tmp = rat.split("/")
    if len(tmp) > 1:
        num = float(tmp[0])
        den = float(tmp[1])
        return num/den
    else:
        return float(tmp[0])

In [3]:
def string_to_list(lista):
    """ Takes a string of rationals on format 1/2 and returns a list of floats """
    splitted = lista.split()
    return [rational_to_double(x) for x in splitted]

In [4]:
def get_payoff_mats(payoffs, p1_n_strats):
    """ Takes a string of payoffs and return an array with the two payoff mats """
    payoff_list = [int(x) for x in payoffs.split()]
    p2_n_strats = int(len(payoff_list)/(2*p1_n_strats))

    p1_payoffs = np.array([payoff_list[i] for i in range(0,len(payoff_list)) if i % 2 == 0])
    p2_payoffs = np.array([payoff_list[i] for i in range(0,len(payoff_list)) if (i+1) % 2 == 0])

    p1_payoff_mat = np.reshape(p1_payoffs, (p1_n_strats, p2_n_strats))
    p2_payoff_mat = np.reshape(p2_payoffs, (p2_n_strats, p1_n_strats), order='F')

    return [p1_payoff_mat, p2_payoff_mat]

In [10]:
# master (1 session), TSE (0-6 sessions), or Mturk (0-4 sessions)
exp = "mturk"
ses = "4"
all_choices = get_clean_data(exp,ses,"choice")
all_players = get_clean_data(exp,ses,"player")
all_games = get_clean_data(exp,ses,"game")
all_payoffs = get_clean_data(exp,ses,"payoff")
all_pasts = get_clean_data(exp,ses,"past")
all_gameplays = get_clean_data(exp,ses,"gameplay")


all_choices["strats"] = all_choices["strats"].apply(string_to_list)
all_payoffs["payoff"] = all_payoffs["payoff"].apply(rational_to_double)
all_pasts["currentsp1"] = all_pasts["currentsp1"].apply(string_to_list)
all_pasts["currentsp2"] = all_pasts["currentsp2"].apply(string_to_list)
all_games["payoffs"] = all_games.apply(lambda row: get_payoff_mats(row["payoffs"], row["p1"]), axis=1)
all_games = all_games.sort_values(['id']).reset_index(drop=True)
all_pasts = all_pasts.sort_values(['round','gameid']).reset_index(drop=True)
#all_pasts["currentsp2"][(all_pasts["round"] == 1) & (all_pasts["gameid"] == 3)][2]
type(all_choices.strats[0][0])

float

In [7]:
init_strats_dict = dict()
init_p1_dict = dict()
init_p2_dict = dict()
payoffs_dict = dict()
role_dict = dict()
id_dict = dict()
actual_plays_dict = dict()

for gid in range(1,len(all_games)+1):
    strats = all_choices[["strats", "playerid"]][(all_choices["round"] == 1) & (all_choices["gameid"] == gid)].copy()
    roles = all_players[["id", "role"]].copy()
    roles["playerid"] = roles["id"]
    pasts = all_pasts[["currentsp1", "currentsp2", "gameid"]][(all_pasts["round"] == 1) & (all_pasts["gameid"] == gid)].copy()

    init_p1_dict = all_pasts["currentsp1"][(all_pasts["round"] == 1) & (all_pasts["gameid"] == gid)].copy()
    init_p2_dict = all_pasts["currentsp2"][(all_pasts["round"] == 1) & (all_pasts["gameid"] == gid)].copy()

    data_mat = strats.set_index("playerid").join(roles.set_index("id"), how='outer').copy()

    # Replace NaN with role average 
    data_mat["strats"][(data_mat["role"] == 0) & pd.isnull(data_mat["strats"])] = [np.repeat(np.array(init_p1_dict), sum((data_mat["role"] == 0) & pd.isnull(data_mat["strats"])))]
    data_mat["strats"][(data_mat["role"] == 1) & pd.isnull(data_mat["strats"])] = [np.repeat(np.array(init_p2_dict), sum((data_mat["role"] == 1) & pd.isnull(data_mat["strats"])))]
    
    plays_vec = [np.array(all_pasts["currentsp1"][all_pasts["gameid"] == gid]), np.array(all_pasts["currentsp2"][all_pasts["gameid"] == gid])]
    plays_vec[0] = np.array([np.array(v) for v in plays_vec[0]])
    plays_vec[1] = np.array([np.array(v) for v in plays_vec[1]])

    #init_strats_dict[gid] = np.array(data_mat["strats"])
    payoffs_dict[gid] = all_games["payoffs"][all_games["id"]==gid][gid-1]
    role_dict[gid] = np.array(data_mat["role"].copy())
    id_dict[gid] = np.array(data_mat["playerid"].copy())
    actual_plays_dict[gid] = plays_vec


C:\Users\peter\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\peter\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\peter\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\peter\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A valu

In [7]:
save_dict = {"init_strats_dict": init_strats_dict, "init_p1_dict":init_p1_dict, 
"init_p2_dict": init_p2_dict, "payoffs_dict":payoffs_dict, 
"role_dict":role_dict, "id_dict":id_dict, "actual_plays_dict":actual_plays_dict}

## Save dictionary 

with open(f"data/data_dicts_{exp}{ses}.pkl", 'wb') as output:
    pickle.dump(save_dict, output, pickle.HIGHEST_PROTOCOL)